In [35]:
#获取索引

import requests
import re
import pandas as pd
import numpy as np
import xlwt
from lxml import etree
import json
gheaders = {
    'Referer': 'https://ieeexplore.ieee.org/xpl/issues?punumber=9&isnumber=9149956',#网站反爬
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'
    }
res = requests.get('https://ieeexplore.ieee.org/rest/publication/9/regular-issues',headers=gheaders)
data = res.text

volume = re.findall('"volume":"(.*?)","issue":".*?".*?,"issueNumber":.*?,.*?"year":".*?","month":".*?"',data)
issue = re.findall('"volume":".*?","issue":"(.*?)".*?,"issueNumber":.*?,.*?"year":".*?","month":".*?"',data)
issuenumber = re.findall('"volume":".*?","issue":".*?".*?,"issueNumber":(.*?),.*?.*?"year":".*?","month":".*?"',data)
year = re.findall('"volume":".*?","issue":".*?".*?,"issueNumber":.*?,.*?"year":"(.*?)","month":".*?"',data)
month = re.findall('"volume":".*?","issue":".*?".*?,"issueNumber":.*?,.*?"year":".*?","month":"(.*?)"',data)

data_all = {
    'volume':np.array(volume),
    'year':np.array(year),
    'month':np.array(month),
    'issue':np.array(issue),
    'issuenumber':np.array(issuenumber)
}
da = pd.DataFrame(data_all)
da

,volume,year,month,issue,issuenumber
0,65,2020,Aug.,8,9149956
1,65,2020,July,7,9127208
2,65,2020,June,6,9103099
3,65,2020,May,5,9078133
4,65,2020,April,4,9049232
...,...,...,...,...,...
559,9,1964,January,1,24303
560,8,1963,October,4,24302
561,8,1963,July,3,24301
562,8,1963,April,2,24300


In [36]:
import requests
import re
import pandas as pd
import numpy as np
import xlwt
from lxml import etree
import json


def find_year(da):
    return da.index.str.startswith("2020")==1

#存储为excel
writer = pd.ExcelWriter(r'D:/jupyter_save/data.xlsx')
da['date'] = da['year'].map(str) + "-" + da['month']
da.set_index('date', inplace = True)
#str.replace('*','*')
da.loc[:,'issuenumber'] = da['issuenumber'].astype('int64')
da.to_excel(writer,sheet_name = 'median_save',index=True)
writer.save()
#test
da.loc[find_year,:]


,volume,year,month,issue,issuenumber
date,,,,,
2020-Aug.,65,2020,Aug.,8,9149956
2020-July,65,2020,July,7,9127208
2020-June,65,2020,June,6,9103099
2020-May,65,2020,May,5,9078133
2020-April,65,2020,April,4,9049232
2020-March,65,2020,March,3,9016278
2020-Feb.,65,2020,Feb.,2,8972645
2020-Jan.,65,2020,Jan.,1,8944139


In [37]:
#数据爬取处理预备
import requests
import re
import pandas as pd
import numpy as np
import xlwt
from lxml import etree
import json


file = pd.ExcelFile(r'./data.xlsx')
handle = pd.read_excel('./data.xlsx')
issuenum = handle['issuenumber']
datestr = handle['date'] 
print('finish')

finish


In [38]:
issuenum

0      9149956
1      9127208
2      9103099
3      9078133
4      9049232
        ...   
559      24303
560      24302
561      24301
562      24300
563      24299
Name: issuenumber, Length: 564, dtype: int64

In [ ]:
from lxml import etree
import json
import time
#id储存在toc数据包
length=len(issuenum)

alf = open(r'save4.txt', 'w')
easy_change = '9'
def get_paperinfo(issuenumber,easy_num):
    jsonerror_flag = 0
    name = pd.Series([0])
    for x in range(450,564):#一次传太多容易报错，改这里，有多少个运行上面第一栏代码可以得到，筛选最新的
        time.sleep(1)
        print(x)
        y = issuenumber.loc[x]
        y = str(y)
        gheaders = {
            'Host': 'ieeexplore.ieee.org',
            'Referer': 'https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber='+y+'&punumber='+easy_num,
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'
            }
        toc_url = 'https://ieeexplore.ieee.org/rest/search/pub/'+easy_num+'/issue/'+y+'/toc'
        for page in range(0,5):
            if page==1:
                payload = '{"isnumber":"'+y+'","punumber":"'+easy_num+'","sortType":"vol-only-seq","pageNumber":"2"}'
            elif page==2:
                payload = '{"isnumber":"'+y+'","punumber":"'+easy_num+'","sortType":"vol-only-seq","pageNumber":"3"}'
            elif page==3:
                payload = '{"isnumber":"'+y+'","punumber":"'+easy_num+'","sortType":"vol-only-seq","pageNumber":"4"}'
            elif page==4:
                payload = '{"isnumber":"'+y+'","punumber":"'+easy_num+'","sortType":"vol-only-seq","pageNumber":"5"}'
            else:
                payload = '{"isnumber":"'+y+'","punumber":"'+easy_num+'","sortType":"vol-only-seq"}'     
            toc_res = requests.post(toc_url, headers = gheaders,data = payload)
            try:
                toc_dic = json.loads(toc_res.text)
                try:
                    articles = toc_dic['records']
                except KeyError:
                    print('不存在第'+str(page+1)+'页')
                    break
                else:
                    for article in articles:
                        flag = article['ephemera']
                        if flag == False:
                            #title = article['highlightedTitle']
                            paperid = article['articleNumber']
                            alf.write(paperid+'\n')
                            print(paperid)
            except json.JSONDecodeError:
                print('JSON error,try again-------issuenumber=%s', y )
                toc_url = 'https://ieeexplore.ieee.org/rest/search/pub/'+easy_num+'/issue/'+y+'/toc'
                payload = '{"isnumber":"'+y+'","punumber":"'+easy_num+'","sortType":"vol-only-seq"}'
                toc_res = requests.post(toc_url, headers = gheaders,data = payload)
                try:
                    toc_dic = json.loads(toc_res.text)
                    try:
                        articles = toc_dic['records']
                    except KeyError:
                        print('KEY_ERROR,filenum=',x)
                        break
                    else:
                        for article in articles:
                            flag = article['ephemera']
                            if flag == False:
                                #title = article['highlightedTitle']
                                paperid = article['articleNumber']
                                alf.write(paperid+'\n')
                                print(paperid)
                except json.JSONDecodeError:
                    name.loc[jsonerror_flag] = y
                    jsonerror_flag = jsonerror_flag + 1
                    print('Error  again,Pass\n')
                    continue
                else:
                    continue
            else:
                continue
    alf.close()
get_paperinfo(issuenum,easy_change)
print('finish')


    



In [44]:
#对得到的id整合
import requests
import re
import pandas as pd
import numpy as np
import xlwt
from lxml import etree
import json


id_data_1 = pd.read_csv(r'./save1.txt',header=None)
id_data_2 = pd.read_csv(r'./save2.txt',header=None)
id_data_3 = pd.read_csv(r'./save3.txt',header=None)
id_data_4 = pd.read_csv(r'./save4.txt',header=None)
# id_data = pd.read_csv(r'./save.txt',header=None)
id_data=pd.concat([id_data_1,id_data_2,id_data_3,id_data_4],axis=0,join = 'inner',ignore_index = True )
id_data.rename(columns={0:'paper-ID'}, inplace = True)
writer_paperid = pd.ExcelWriter(r'D:/jupyter_save/paperid.xlsx')
id_data.to_excel(writer_paperid,sheet_name = 'paper_idsave',index=True)
writer_paperid.save()

print('Finish')

Finish


In [45]:
#对所有文章进行信息获取
import requests
import re
import pandas as pd
import numpy as np
import xlwt
from lxml import etree
import json

id_handle = pd.read_excel(r'./paperid.xlsx')
id_data = id_handle['paper-ID']
len(id_data)

17598

In [10]:
#对所有文章进行信息获取
import requests
import re
import pandas as pd
import numpy as np
import xlwt
from lxml import etree
import json

id_handle = pd.read_excel(r'./paperid.xlsx')
id_data = id_handle['paper-ID']
#id_data
info_creat = pd.DataFrame({'Year':[],
                          'Month':[],
                          'Day':[],
                          'Name':[],
                          'Keyword_0':[],
                          'Keyword_1':[],
                          'Keyword_2':[],
                          'Keyword_3':[],
                          'Keyword_4':[]                                                       
                         })
info_save = info_creat
def get_info(id_data, info_save,info_creat):
    finish_flag = False
    max_index = 17598
    median_info = info_creat
    print('一组20篇文章，组数如下')
    for x in range(850,900):
        times = x+1
        if finish_flag == False:
            print(times)
            for y in range(0,20):
                flag = x*20 + y
                if flag < max_index:
                    id_test = id_data.loc[flag]
                    id_test = str(id_test)
                    gheaders = {
                    'Host': 'ieeexplore.ieee.org',
                    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
                    'Referer': 'https://ieeexplore.ieee.org/document/'+id_test+'/keywords',
                    }
                    metadata_url='https://ieeexplore.ieee.org/document/'+id_test+'/keywords'
                    #try看情况是否要添加
                    meta_res = requests.get(metadata_url,headers=gheaders)
                    info_data = meta_res.text
                    #global.document.metadata.*?"Keywords","kwd":(.*?)}],"isGetArticle".*?"journalDisplayDateOfPublication":.*?,"isPromo"
                    keywords = re.findall('"IEEE Keywords","kwd":(.*?)]}',info_data)
                    pub=re.findall('"journalDisplayDateOfPublication":(.*?),',info_data)
                    paper=re.findall('<title>(.*?) - IEEE',info_data)
                    keyword=''.join(keywords)
                    paper_name=''.join(paper)
                    pub=''.join(pub)
                    pub_date=pub.replace('"','')
                    keyword=keyword.replace('"','')
                    keyword=keyword.replace('[','')
                    keyword=keyword.replace(']','')
                    keyword_all=keyword.split(',')
                    pub_date=pub_date.split(' ')
                    if len(pub_date) == 3:
                        table_creat = pd.DataFrame({'Year':[pub_date[2]],
                                                    'Month':[pub_date[1]],
                                                    'Day':[pub_date[0]],
                                                    'Name':[paper_name],                                                   
                                                   })
                        for i in range(0,len(keyword_all)):
                            name_i = str(i)
                            table_creat.loc[:,"Keyword_"+name_i]=[keyword_all[i]]
                            #备忘：这里是当前文章的一行信息
                        median_info = pd.concat([median_info,table_creat],ignore_index = True)
                        #备忘：median_info循环20遍记录数据
                    elif len(pub_date)==1:
                        pub_date.insert(0,'00')
                        pub_date.insert(0,'00')
                        table_creat = pd.DataFrame({'Year':[pub_date[2]],
                                                    'Month':[pub_date[1]],
                                                    'Day':[pub_date[0]],
                                                    'Name':[paper_name],                                                   
                                                   })
                        for i in range(0,len(keyword_all)):
                            name_i = str(i)
                            table_creat.loc[:,"Keyword_"+name_i]=[keyword_all[i]]
                            #备忘：这里是当前文章的一行信息
                        median_info = pd.concat([median_info,table_creat],ignore_index = True)
                        #备忘：median_info循环20遍记录数据
                    else:
                        pub_date.insert(0,'00')
                        table_creat = pd.DataFrame({'Year':[pub_date[2]],
                                                    'Month':[pub_date[1]],
                                                    'Day':[pub_date[0]],
                                                    'Name':[paper_name],                                                   
                                                   })
                        for i in range(0,len(keyword_all)):
                            name_i = str(i)
                            table_creat.loc[:,"Keyword_"+name_i]=[keyword_all[i]]
                            #备忘：这里是当前文章的一行信息
                        median_info = pd.concat([median_info,table_creat],ignore_index = True)
                        #备忘：median_info循环20遍记录数据
                else:
                    finish_flag = True
                    break
            info_save = pd.concat([info_save,median_info],ignore_index = True)
            median_info = info_creat
            print('第'+str(times)+'组完成')
        else:            
            print('全部完成,无剩余内容')
            break
    return info_save 
info_data = get_info(id_data, info_save,info_creat)
paper_info = pd.ExcelWriter(r'D:/jupyter_save/paper_info.xlsx')
info_data.to_excel(paper_info,sheet_name = 'median_file',index=False)
paper_info.save()
runned_flag = True
print('Finish')

一组20篇文章，组数如下
851
第851组完成
852
第852组完成
853
第853组完成
854
第854组完成
855
第855组完成
856
第856组完成
857
第857组完成
858
第858组完成
859
第859组完成
860
第860组完成
861
第861组完成
862
第862组完成
863
第863组完成
864
第864组完成
865
第865组完成
866
第866组完成
867
第867组完成
868
第868组完成
869
第869组完成
870
第870组完成
871
第871组完成
872
第872组完成
873
第873组完成
874
第874组完成
875
第875组完成
876
第876组完成
877
第877组完成
878
第878组完成
879
第879组完成
880
第880组完成
全部完成,无剩余内容
Finish


In [1]:
runned_flag

NameError: name 'runned_flag' is not defined

In [47]:
# #建立此期刊对应sheet且会初始化！！！
# import requests
# import re
# import pandas as pd
# import numpy as np
# import xlwt
# from lxml import etree
# import json

# This = pd.DataFrame({'Year':[],
#                           'Month':[],
#                           'Day':[],
#                           'Name':[],
#                           'Keyword_0':[],
#                           'Keyword_1':[],
#                           'Keyword_2':[],
#                           'Keyword_3':[],
#                           'Keyword_4':[]                                                       
#                          })
# writer = pd.ExcelWriter(r'D:/jupyter_save/total_info.xlsx')
# This.to_excel(writer,sheet_name = 'save',index=False)
# writer.save()
# print('Finish')


Finish


In [ ]:
info_data

In [11]:
#分批录入

import requests
import re
import pandas as pd
import numpy as np
import xlwt
from lxml import etree
import json
if runned_flag ==True:
    merged_writer = pd.ExcelWriter(r'D:/jupyter_save/total_info.xlsx')
    merge_info_now = pd.read_excel(r'./paper_info.xlsx')
    merge_info_pro = pd.read_excel(r'./total_info.xlsx')
    merge_allinfo = pd.concat([merge_info_pro,merge_info_now],ignore_index = True)
    merge_allinfo.to_excel(merged_writer,sheet_name = 'save',index=False)
    merged_writer.save()
    runned_flag =False
    print('Finish')
else:
    print('无需重复录入')

Finish


In [12]:
#统一不同时期的格式并存储
import requests
import re
import pandas as pd
import numpy as np
import xlwt
from lxml import etree
import json
import openpyxl


def add_sheet(data, excel_writer, sheet_name):
    book = openpyxl.load_workbook(excel_writer.path)
    excel_writer.book = book
    data.to_excel(excel_writer=excel_writer, sheet_name=sheet_name, index=True, header=True)
    excel_writer.close()
    
    
    
writer = pd.ExcelWriter(r'D:/jupyter_save/Final_info.xlsx', engine='openpyxl')
info = pd.read_excel(r'./total_info.xlsx')
info['Date'] = info['Month'].map(str)  + info['Day'].map(str)
info_fix = info['Date'].str.replace("[0123456789.]","").str.replace('nan','')
info_fix = info_fix.str.replace('Jan','01').str.replace('Feb','02').str.replace('Mar','03').str.replace('Apr','04').str.replace('May','05').str.replace('Nov','11')
info_fix = info_fix.str.replace('Jun','06').str.replace('Jul','07').str.replace('Aug','08').str.replace('Sep','09').str.replace('Oct','10').str.replace('Dec','12')
info_fix = info_fix.str.replace("[abcehilmorstuy]","")
info['Date'] = info_fix
info['Date'] = info['Year'].map(str)  +'-'+ info['Date'].map(str)
# info['Date'] = info['Date'].str.replace('\n','')
del info['Month']
del info['Day']
del info['Year']
info.set_index('Date',inplace = True)

add_sheet(info, writer, 'IEEE Trans on Automatic Control')
# info.to_excel(writer,sheet_name = 'IEEE Transactions on Networking',index=True)
# writer.save()
print('finish')


finish
